In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
import pandas as pd
from tqdm import tqdm
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter, defaultdict

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
# Load the TSV file into a DataFrame
vocab_1 = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/Technet/vocab_github_1.tsv', sep='\t', header=None)
vocab_1 = vocab_1.rename(columns={0: "word"})
# Display the contents
vocab_1

,word,1,2
0,method,0,0
1,form,0,1
2,least,0,2
3,comprise,0,3
4,system,0,4
...,...,...,...
1999995,abe_fermentation,199,9995
1999996,configurates,199,9996
1999997,galactinol,199,9997
1999998,cloned_leptospira_outer_membrane,199,9998


In [4]:
# Load the TSV file into a DataFrame
vocab_2 = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/Technet/vocab_github_2.tsv', sep='\t', header=None)

vocab_2= vocab_2.rename(columns={0: "word"})
# Display the contents
vocab_2

,word,1,2
0,fourth_lens_b54/b,200,0
1,compact_fluid,200,1
2,triphenylmethane_xanthene_thioxanthone_nitroaryl,200,2
3,slotted-ring,200,3
4,box_bottom_14b,200,4
...,...,...,...
2038919,compensation/control,403,8919
2038920,neighboring_claw,403,8920
2038921,reinforced_elastomer_section,403,8921
2038922,engaging_portion_b211/b,403,8922


In [5]:
v1_woNA = vocab_1.dropna(subset=['word'])  #vocab_1[vocab_1['word'].str.contains(r'^[^\s]+(?:_[^\s]+)*$', na=False)] #.str.contains(r'^[^\s_]+$', na=False)]
v1_woNA

,word,1,2
0,method,0,0
1,form,0,1
2,least,0,2
3,comprise,0,3
4,system,0,4
...,...,...,...
1999995,abe_fermentation,199,9995
1999996,configurates,199,9996
1999997,galactinol,199,9997
1999998,cloned_leptospira_outer_membrane,199,9998


In [6]:
print(v1_woNA.equals(vocab_1)) # NaN values!
vocab_1['word']
# v1_1word["word"]
vocab_1[~vocab_1["word"].isin(v1_woNA['word'])]

False


,word,1,2
1064,NaN,0,1064
1433,NaN,0,1433
7798,NaN,0,7798


In [7]:
v2_woNA = vocab_2.dropna(subset=['word']) #vocab_2[vocab_2["word"].str.contains(r'^[^\s]+(?:_[^\s]+)*$', na=False)] #.str.contains(r'^[^\s_]+$', na=False)]
v2_woNA

,word,1,2
0,fourth_lens_b54/b,200,0
1,compact_fluid,200,1
2,triphenylmethane_xanthene_thioxanthone_nitroaryl,200,2
3,slotted-ring,200,3
4,box_bottom_14b,200,4
...,...,...,...
2038919,compensation/control,403,8919
2038920,neighboring_claw,403,8920
2038921,reinforced_elastomer_section,403,8921
2038922,engaging_portion_b211/b,403,8922


In [8]:
def split_underscored_words(word):
    # if ' ' not in word:  # Keep only words without spaces
    return word.split('_')  # Split on underscores
    # else:
        # return []

# Apply the function to the 'word' column and flatten the result
# split_words = v1_1word['word'].apply(split_underscored_words)

def flatten(split_words):
    flat_list = [item for sublist in split_words for item in sublist]
    unique_words = set(flat_list)
    # Create a new DataFrame with the split words
    df_split = pd.DataFrame(sorted(unique_words), columns=['word'])
    return(df_split)

def splitFlatten(vocab):
    split_words = vocab['word'].apply(split_underscored_words)
    return(flatten(split_words))

In [9]:
v1v2 = pd.DataFrame(sorted(set(pd.DataFrame(pd.concat([splitFlatten(v1_woNA), splitFlatten(v2_woNA)], axis=0)["word"])["word"])))
v1v2.rename(columns={0: 'word'}, inplace=True)
v1v2

,word
0,
1,-
2,/
3,/-0002
4,/-01
...,...
845999,zzero
846000,zzn
846001,zzp
846002,zzu


In [10]:
technet = pd.read_csv("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/Technet/clean_vocab.csv")
technet

,technet_vocab
0,x394fo
1,14-substituted-21h-quinazolinone
2,toggle-linkage
3,anolyte/catholyte
4,flammability
...,...
845998,siloxide
845999,backpropagated
846000,beidellite
846001,hdpcvd


In [11]:
print(v1v2[~v1v2['word'].isin(technet['technet_vocab'])])
v1v2_test = pd.DataFrame(sorted(v1v2.drop(list(v1v2[~v1v2['word'].isin(technet['technet_vocab'])]["word"].index))["word"]))
v1v2_test

        word
0           
498846   nan
528277  null


,0
0,-
1,/
2,/-0002
3,/-01
4,/-1/16fsub0
...,...
845996,zzero
845997,zzn
845998,zzp
845999,zzu


In [12]:
print(technet[~technet['technet_vocab'].isin(v1v2['word'])])
technet_test = pd.DataFrame(sorted(technet.drop(list(technet[~technet['technet_vocab'].isin(v1v2['word'])].index))["technet_vocab"]))
technet_test

       technet_vocab
123078           NaN
678532           NaN


,0
0,-
1,/
2,/-0002
3,/-01
4,/-1/16fsub0
...,...
845996,zzero
845997,zzn
845998,zzp
845999,zzu


In [13]:
technet_test.equals(v1v2_test)

True

En retirant les nan, null, NaN, et vide, on obtient des dataframes identiques.

Bref, le technet utilisé:
- séparer par underscores

Comme il s'agit d'un dictionnaire, on garde tous les mots sans lemmatizer.

Nous allons maintenant lemmatizer le vocabulaire, à l'aide de spaCy

In [14]:
# retirer NA
technet_NA=technet.dropna(subset=['technet_vocab']).reset_index(drop=True)

In [15]:
print(list(technet['technet_vocab'][123077:123080]))
print(list(technet_NA['technet_vocab'][123077:123080]))

['opposite-light-emitter-side', nan, 'pre-causticisation']
['opposite-light-emitter-side', 'pre-causticisation', 'i-fold']


In [16]:
set_vocab = list(technet_NA['technet_vocab'])
lemmatized_technet = []

# Use nlp.pipe() for batch processing
for doc in tqdm(nlp.pipe(set_vocab, disable=["ner", "parser"])):
    # Join token lemmas, preserving dashes/slashes if necessary
    lemmatized_word = "".join([token.lemma_ if token.is_alpha else token.text for token in doc])
    lemmatized_technet.append(lemmatized_word)

846001it [05:17, 2668.03it/s]


In [17]:
len(list(set(lemmatized_technet)))

778702

In [29]:
lemmatized_technet=pd.DataFrame(lemmatized_technet, columns=["lemmatized"])

In [31]:
df=pd.concat([pd.DataFrame(technet_NA), pd.DataFrame(lemmatized_technet, columns=["lemmatized"])], axis=1)
df[df['technet_vocab'] != df["lemmatized"]]

,technet_vocab,lemmatized
7,probe-mounted,probe-mount
13,polynucelotides,polynucelotide
21,pedal-assisted,pedal-assist
26,needlesticks,needlestick
28,shocklines,shockline
...,...,...
845971,set/controlled,set/control
845975,pancreas,pancrea
845990,humidity-compromised,humidity-compromise
845991,subcollections,subcollection


In [33]:
df.to_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/Technet/lemmatized_technet.csv', index=False)